# Centrality Measures in Cybersecurity 🔒

Welcome to this comprehensive tutorial on centrality measures and their applications in cybersecurity! In this notebook, we'll explore how graph theory can be used to analyze network topologies, identify critical infrastructure, and detect vulnerabilities.

## Concepts Covered:
- **Centrality Measures**: Degree, Betweenness, Eigenvector, PageRank, Closeness
- **Single Points of Failure (SPOF)**: Critical nodes and edges that could disrupt the network
- **Cut-Vertices (Articulation Points)**: Nodes whose removal disconnects the network
- **Bridges**: Edges whose removal disconnects the network
- **Tarjan's Algorithm**: Efficient algorithm for finding cut-vertices and bridges
- **Cybersecurity Applications**: Network security analysis, attack path identification, defense strategies

## What You'll Learn:
1. How to compute various centrality measures on network graphs
2. How to identify critical infrastructure using graph analysis
3. How hackers can use centrality to identify attack targets
4. How blue teams can use centrality to protect their networks
5. Visualizing network vulnerabilities and critical paths


In [ ]:
import warnings
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
from collections import defaultdict, deque
import random

warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

print("✅ All libraries imported successfully!")

## Part I: Generating a Realistic Corporate Network

We'll create a realistic corporate network topology that represents:
- **Core Infrastructure**: Main routers, switches, firewalls
- **Departments**: IT, Finance, HR, Marketing, Engineering
- **Servers**: Database servers, web servers, file servers
- **Workstations**: Employee computers
- **Network Devices**: Printers, IoT devices, access points

This network will have enough complexity to demonstrate various centrality measures and vulnerabilities.


In [ ]:
def generate_corporate_network():
    """
    Generate a realistic corporate network topology.
    Returns a NetworkX graph representing a company's network infrastructure.
    """
    G = nx.Graph()
    
    # Core infrastructure (highly connected)
    core_devices = [
        'Core-Router-01', 'Core-Router-02', 'Core-Switch-01', 
        'Core-Switch-02', 'Firewall-01', 'Firewall-02',
        'DMZ-Gateway', 'Internet-Gateway'
    ]
    
    # Departments
    departments = {
        'IT': ['IT-Server-01', 'IT-Server-02', 'IT-Workstation-01', 'IT-Workstation-02', 
               'IT-Workstation-03', 'IT-Switch-01', 'IT-Printer-01'],
        'Finance': ['Finance-Server-01', 'Finance-DB-01', 'Finance-Workstation-01', 
                   'Finance-Workstation-02', 'Finance-Workstation-03', 'Finance-Switch-01'],
        'HR': ['HR-Server-01', 'HR-Workstation-01', 'HR-Workstation-02', 
               'HR-Workstation-03', 'HR-Switch-01'],
        'Marketing': ['Marketing-Server-01', 'Marketing-Workstation-01', 
                     'Marketing-Workstation-02', 'Marketing-Workstation-03', 
                     'Marketing-Switch-01', 'Marketing-Printer-01'],
        'Engineering': ['Eng-Server-01', 'Eng-Server-02', 'Eng-DB-01', 
                       'Eng-Workstation-01', 'Eng-Workstation-02', 'Eng-Workstation-03',
                       'Eng-Workstation-04', 'Eng-Switch-01', 'Eng-Build-Server-01'],
        'Executive': ['Exec-Server-01', 'Exec-Workstation-01', 'Exec-Workstation-02', 
                     'Exec-Switch-01'],
        'Sales': ['Sales-Server-01', 'Sales-Workstation-01', 'Sales-Workstation-02',
                 'Sales-Workstation-03', 'Sales-Switch-01']
    }
    
    # Critical servers (high value targets)
    critical_servers = [
        'Domain-Controller-01', 'Domain-Controller-02', 'Email-Server-01',
        'Backup-Server-01', 'File-Server-01', 'Web-Server-01', 'Web-Server-02',
        'Database-Cluster-01', 'Database-Cluster-02'
    ]
    
    # Network devices
    network_devices = [
        'Access-Point-01', 'Access-Point-02', 'Access-Point-03',
        'VPN-Gateway-01', 'Load-Balancer-01', 'Proxy-Server-01'
    ]
    
    # Add all nodes with attributes
    G.add_nodes_from(core_devices, node_type='core', criticality='high')
    G.add_nodes_from(critical_servers, node_type='server', criticality='critical')
    
    for dept, devices in departments.items():
        for device in devices:
            if 'Server' in device or 'DB' in device:
                G.add_node(device, node_type='server', department=dept, criticality='high')
            elif 'Workstation' in device:
                G.add_node(device, node_type='workstation', department=dept, criticality='medium')
            elif 'Switch' in device:
                G.add_node(device, node_type='switch', department=dept, criticality='high')
            else:
                G.add_node(device, node_type='device', department=dept, criticality='low')
    
    G.add_nodes_from(network_devices, node_type='network', criticality='medium')
    
    # Create core network (highly connected mesh)
    for i, node1 in enumerate(core_devices):
        for node2 in core_devices[i+1:]:
            if random.random() < 0.6:  # 60% connectivity in core
                G.add_edge(node1, node2, weight=1, link_type='core')
    
    # Connect core to critical servers
    for server in critical_servers:
        # Each critical server connects to 2-3 core devices
        connected_cores = random.sample(core_devices, min(3, len(core_devices)))
        for core in connected_cores:
            G.add_edge(server, core, weight=2, link_type='server-core')
    
    # Connect departments to core
    for dept, devices in departments.items():
        dept_switches = [d for d in devices if 'Switch' in d]
        dept_servers = [d for d in devices if 'Server' in d or 'DB' in d]
        dept_workstations = [d for d in devices if 'Workstation' in d]
        dept_other = [d for d in devices if d not in dept_switches + dept_servers + dept_workstations]
        
        # Connect department switch to core
        if dept_switches:
            dept_switch = dept_switches[0]
            # Connect to 2-3 core devices
            connected_cores = random.sample(core_devices, min(3, len(core_devices)))
            for core in connected_cores:
                G.add_edge(dept_switch, core, weight=2, link_type='dept-core')
        
        # Connect servers to department switch or directly to core
        for server in dept_servers:
            if dept_switches and random.random() < 0.7:
                G.add_edge(server, dept_switches[0], weight=1, link_type='server-switch')
            else:
                # Direct connection to core
                core = random.choice(core_devices)
                G.add_edge(server, core, weight=2, link_type='server-core')
        
        # Connect workstations to department switch
        if dept_switches:
            for workstation in dept_workstations:
                G.add_edge(workstation, dept_switches[0], weight=1, link_type='workstation-switch')
        
        # Connect other devices
        for device in dept_other:
            if dept_switches:
                G.add_edge(device, dept_switches[0], weight=1, link_type='device-switch')
    
    # Connect network devices to core
    for device in network_devices:
        if 'VPN' in device or 'Gateway' in device:
            # VPN and gateways connect to multiple core devices
            connected_cores = random.sample(core_devices, min(2, len(core_devices)))
            for core in connected_cores:
                G.add_edge(device, core, weight=2, link_type='network-core')
        else:
            # Other network devices connect to one core device
            core = random.choice(core_devices)
            G.add_edge(device, core, weight=2, link_type='network-core')
    
    # Add some inter-department connections (lower probability)
    all_switches = [n for n in G.nodes() if 'Switch' in n]
    for i, switch1 in enumerate(all_switches):
        for switch2 in all_switches[i+1:]:
            if random.random() < 0.15:  # 15% chance of inter-department connection
                G.add_edge(switch1, switch2, weight=3, link_type='inter-dept')
    
    # Add some workstation-to-workstation connections (peer-to-peer, lower probability)
    all_workstations = [n for n in G.nodes() if 'Workstation' in n]
    for _ in range(len(all_workstations) // 10):  # ~10% of workstations have peer connections
        ws1, ws2 = random.sample(all_workstations, 2)
        if not G.has_edge(ws1, ws2):
            G.add_edge(ws1, ws2, weight=5, link_type='peer')
    
    # Add attributes to edges if not present
    for u, v in G.edges():
        if 'weight' not in G[u][v]:
            G[u][v]['weight'] = 1
    
    return G

# Generate the network
print("Generating corporate network...")
G = generate_corporate_network()

print(f"\n✅ Network generated successfully!")
print(f"   Nodes: {G.number_of_nodes()}")
print(f"   Edges: {G.number_of_edges()}")
print(f"   Density: {nx.density(G):.4f}")
print(f"   Is connected: {nx.is_connected(G)}")
print(f"   Number of components: {nx.number_connected_components(G)}")

# Display node type distribution
node_types = [G.nodes[n].get('node_type', 'unknown') for n in G.nodes()]
type_counts = pd.Series(node_types).value_counts()
print(f"\n📊 Node type distribution:")
for node_type, count in type_counts.items():
    print(f"   {node_type}: {count}")


## Part II: Centrality Measures

Centrality measures help identify the most important nodes in a network. In cybersecurity, these measures can reveal:
- **High-value targets** for attackers
- **Critical infrastructure** that needs protection
- **Bottlenecks** that could cause network disruption
- **Influence propagation** paths for malware or attacks


In [ ]:
# Compute all centrality measures
print("Computing centrality measures...")

# Degree Centrality: Number of connections
degree_centrality = nx.degree_centrality(G)

# Betweenness Centrality: How often a node lies on the shortest path between other nodes
betweenness_centrality = nx.betweenness_centrality(G, weight='weight')

# Closeness Centrality: How close a node is to all other nodes
closeness_centrality = nx.closeness_centrality(G, distance='weight')

# Eigenvector Centrality: Importance based on connections to important nodes
eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000, weight='weight')

# PageRank: Google's algorithm for ranking nodes
pagerank = nx.pagerank(G, weight='weight')

# Create a DataFrame with all centrality measures
centrality_df = pd.DataFrame({
    'node': list(G.nodes()),
    'degree': [degree_centrality.get(n, 0) for n in G.nodes()],  # Normalized degree centrality
    'degree_count': [G.degree(n) for n in G.nodes()],  # Actual degree count
    'betweenness': [betweenness_centrality.get(n, 0) for n in G.nodes()],
    'closeness': [closeness_centrality.get(n, 0) for n in G.nodes()],
    'eigenvector': [eigenvector_centrality.get(n, 0) for n in G.nodes()],
    'pagerank': [pagerank.get(n, 0) for n in G.nodes()],
    'node_type': [G.nodes[n].get('node_type', 'unknown') for n in G.nodes()],
    'department': [G.nodes[n].get('department', 'N/A') for n in G.nodes()],
    'criticality': [G.nodes[n].get('criticality', 'medium') for n in G.nodes()]
})

# Sort by betweenness centrality (often most revealing for network topology)
centrality_df = centrality_df.sort_values('betweenness', ascending=False)

print("✅ Centrality measures computed!")
print(f"\n📊 Top 10 nodes by Betweenness Centrality (network bottlenecks):")
print(centrality_df[['node', 'betweenness', 'degree_count', 'node_type', 'criticality']].head(10).to_string(index=False))


In [ ]:
def plot_network_centrality(G, centrality_dict, title, node_size_multiplier=1000, figsize=(16, 12)):
    """
    Plot network with nodes sized and colored by centrality measure.
    """
    # Compute layout
    pos = nx.spring_layout(G, k=2, iterations=50, seed=42)
    
    # Prepare node sizes and colors
    node_sizes = [centrality_dict.get(node, 0) * node_size_multiplier + 100 for node in G.nodes()]
    node_colors = [centrality_dict.get(node, 0) for node in G.nodes()]
    
    # Get node types for coloring
    node_types = [G.nodes[node].get('node_type', 'unknown') for node in G.nodes()]
    type_to_color = {
        'core': '#FF6B6B',      # Red
        'server': '#4ECDC4',    # Teal
        'workstation': '#95E1D3', # Light teal
        'switch': '#F38181',    # Pink
        'network': '#AA96DA',   # Purple
        'device': '#FCBAD3'     # Light pink
    }
    node_colors_type = [type_to_color.get(nt, '#CCCCCC') for nt in node_types]
    
    # Create figure
    plt.figure(figsize=figsize)
    
    # Draw edges
    nx.draw_networkx_edges(G, pos, alpha=0.2, width=0.5, edge_color='gray')
    
    # Draw nodes by type (colored) and centrality (sized)
    for node_type in set(node_types):
        nodes_of_type = [node for node in G.nodes() if G.nodes[node].get('node_type') == node_type]
        node_sizes_type = [centrality_dict.get(node, 0) * node_size_multiplier + 100 for node in nodes_of_type]
        nx.draw_networkx_nodes(
            G, pos,
            nodelist=nodes_of_type,
            node_size=node_sizes_type,
            node_color=type_to_color.get(node_type, '#CCCCCC'),
            alpha=0.8,
            label=node_type
        )
    
    # Draw labels for important nodes (top 10% by centrality)
    sorted_nodes = sorted(centrality_dict.items(), key=lambda x: x[1], reverse=True)
    top_nodes = [node for node, _ in sorted_nodes[:len(G.nodes())//10]]
    labels = {node: node for node in top_nodes}
    nx.draw_networkx_labels(G, pos, labels, font_size=8, font_weight='bold')
    
    plt.title(title, fontsize=16, fontweight='bold', pad=20)
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Plot with different centrality measures
print("📊 Visualizing network with Degree Centrality...")
plot_network_centrality(G, degree_centrality, 
                       "Network Topology - Node Size = Degree Centrality\n(Larger nodes have more connections)",
                       node_size_multiplier=2000)


In [ ]:
print("📊 Visualizing network with Betweenness Centrality...")
plot_network_centrality(G, betweenness_centrality,
                       "Network Topology - Node Size = Betweenness Centrality\n(Larger nodes are critical bottlenecks)",
                       node_size_multiplier=5000)

### 2.2 Comparing Centrality Measures

Let's compare different centrality measures to understand which nodes are critical from different perspectives.


In [ ]:
# Create a comparison visualization
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=('Degree Centrality', 'Betweenness Centrality', 'Closeness Centrality',
                   'Eigenvector Centrality', 'PageRank', 'Centrality Comparison'),
    specs=[[{"type": "bar"}, {"type": "bar"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "bar"}, {"type": "scatter"}]]
)

# Get top 15 nodes by each measure
top_n = 15

# Degree Centrality
top_degree = centrality_df.nlargest(top_n, 'degree')
fig.add_trace(
    go.Bar(x=top_degree['node'], y=top_degree['degree'], name='Degree',
           marker_color='#FF6B6B', text=top_degree['degree'].round(3),
           textposition='outside'),
    row=1, col=1
)

# Betweenness Centrality
top_betweenness = centrality_df.nlargest(top_n, 'betweenness')
fig.add_trace(
    go.Bar(x=top_betweenness['node'], y=top_betweenness['betweenness'], name='Betweenness',
           marker_color='#4ECDC4', text=top_betweenness['betweenness'].round(3),
           textposition='outside'),
    row=1, col=2
)

# Closeness Centrality
top_closeness = centrality_df.nlargest(top_n, 'closeness')
fig.add_trace(
    go.Bar(x=top_closeness['node'], y=top_closeness['closeness'], name='Closeness',
           marker_color='#95E1D3', text=top_closeness['closeness'].round(3),
           textposition='outside'),
    row=1, col=3
)

# Eigenvector Centrality
top_eigenvector = centrality_df.nlargest(top_n, 'eigenvector')
fig.add_trace(
    go.Bar(x=top_eigenvector['node'], y=top_eigenvector['eigenvector'], name='Eigenvector',
           marker_color='#F38181', text=top_eigenvector['eigenvector'].round(3),
           textposition='outside'),
    row=2, col=1
)

# PageRank
top_pagerank = centrality_df.nlargest(top_n, 'pagerank')
fig.add_trace(
    go.Bar(x=top_pagerank['node'], y=top_pagerank['pagerank'], name='PageRank',
           marker_color='#AA96DA', text=top_pagerank['pagerank'].round(4),
           textposition='outside'),
    row=2, col=2
)

# Scatter plot comparing betweenness vs degree
fig.add_trace(
    go.Scatter(
        x=centrality_df['degree'],
        y=centrality_df['betweenness'],
        mode='markers+text',
        text=centrality_df['node'],
        textposition="top center",
        textfont=dict(size=8),
        marker=dict(
            size=10,
            color=centrality_df['pagerank'],
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title="PageRank", x=1.15)
        ),
        name='Nodes',
        hovertemplate='<b>%{text}</b><br>Degree: %{x:.3f}<br>Betweenness: %{y:.3f}<extra></extra>'
    ),
    row=2, col=3
)

# Update x-axis labels
for i in range(1, 3):
    for j in range(1, 4):
        fig.update_xaxes(tickangle=-45, row=i, col=j)

# Update layout
fig.update_layout(
    height=1000,
    showlegend=False,
    title_text="Centrality Measures Comparison",
    title_x=0.5
)

fig.update_xaxes(title_text="Node", row=2, col=3)
fig.update_yaxes(title_text="Betweenness", row=2, col=3)
fig.update_xaxes(title_text="Degree", row=2, col=3)

fig.show()

## Part III: Single Points of Failure (SPOF) Detection

In cybersecurity, identifying Single Points of Failure (SPOF) is crucial. A SPOF is a component whose failure would cause the entire system to fail. In network graphs, these are:
- **Cut-vertices (Articulation Points)**: Nodes whose removal disconnects the network
- **Bridges**: Edges whose removal disconnects the network

### Tarjan's Algorithm

Tarjan's algorithm is an efficient algorithm for finding cut-vertices and bridges in a graph. It uses Depth-First Search (DFS) and runs in O(V + E) time.


In [ ]:
def tarjan_cut_vertices(G):
    """
    Find all articulation points (cut-vertices) in the graph using Tarjan's algorithm.
    An articulation point is a vertex whose removal increases the number of connected components.
    
    Returns:
        set: Set of articulation points
    """
    if not nx.is_connected(G):
        # For disconnected graphs, find articulation points in each component
        articulation_points = set()
        for component in nx.connected_components(G):
            subgraph = G.subgraph(component)
            articulation_points.update(tarjan_cut_vertices_connected(subgraph))
        return articulation_points
    
    return tarjan_cut_vertices_connected(G)

def tarjan_cut_vertices_connected(G):
    """
    Find articulation points in a connected graph using Tarjan's algorithm.
    """
    articulation_points = set()
    discovery = {}
    low = {}
    parent = {}
    time = [0]  # Use list to allow modification in nested function
    
    def dfs(u):
        discovery[u] = time[0]
        low[u] = time[0]
        time[0] += 1
        children = 0
        
        for v in G.neighbors(u):
            if v not in discovery:
                parent[v] = u
                children += 1
                dfs(v)
                
                # Update low value of u
                low[u] = min(low[u], low[v])
                
                # u is an articulation point if:
                # 1. It's the root and has more than one child
                # 2. It's not the root and low[v] >= discovery[u]
                if parent.get(u) is None and children > 1:
                    articulation_points.add(u)
                if parent.get(u) is not None and low[v] >= discovery[u]:
                    articulation_points.add(u)
            elif v != parent.get(u):
                # Update low value of u for back edge
                low[u] = min(low[u], discovery[v])
    
    # Start DFS from first node
    if G.number_of_nodes() > 0:
        start_node = list(G.nodes())[0]
        parent[start_node] = None
        dfs(start_node)
    
    return articulation_points

def tarjan_bridges(G):
    """
    Find all bridges (cut-edges) in the graph using Tarjan's algorithm.
    A bridge is an edge whose removal increases the number of connected components.
    
    Returns:
        set: Set of bridges (as tuples (u, v))
    """
    if not nx.is_connected(G):
        # For disconnected graphs, find bridges in each component
        bridges = set()
        for component in nx.connected_components(G):
            subgraph = G.subgraph(component)
            bridges.update(tarjan_bridges_connected(subgraph))
        return bridges
    
    return tarjan_bridges_connected(G)

def tarjan_bridges_connected(G):
    """
    Find bridges in a connected graph using Tarjan's algorithm.
    """
    bridges = set()
    discovery = {}
    low = {}
    parent = {}
    time = [0]
    
    def dfs(u):
        discovery[u] = time[0]
        low[u] = time[0]
        time[0] += 1
        
        for v in G.neighbors(u):
            if v not in discovery:
                parent[v] = u
                dfs(v)
                
                # Update low value of u
                low[u] = min(low[u], low[v])
                
                # If low[v] > discovery[u], then (u, v) is a bridge
                if low[v] > discovery[u]:
                    # Add edge in canonical form (smaller node first)
                    bridges.add(tuple(sorted([u, v])))
            elif v != parent.get(u):
                # Update low value of u for back edge
                low[u] = min(low[u], discovery[v])
    
    # Start DFS from first node
    if G.number_of_nodes() > 0:
        start_node = list(G.nodes())[0]
        parent[start_node] = None
        dfs(start_node)
    
    return bridges

# Find cut-vertices and bridges
print("🔍 Analyzing network for Single Points of Failure...")
cut_vertices = tarjan_cut_vertices(G)
bridges = tarjan_bridges(G)

print(f"\n✅ Analysis complete!")
print(f"   Cut-vertices (Articulation Points): {len(cut_vertices)}")
print(f"   Bridges (Cut-edges): {len(bridges)}")

if cut_vertices:
    print(f"\n⚠️  CRITICAL: Found {len(cut_vertices)} articulation points:")
    for i, vertex in enumerate(sorted(cut_vertices), 1):
        node_type = G.nodes[vertex].get('node_type', 'unknown')
        dept = G.nodes[vertex].get('department', 'N/A')
        criticality = G.nodes[vertex].get('criticality', 'medium')
        print(f"   {i}. {vertex} (Type: {node_type}, Department: {dept}, Criticality: {criticality})")

if bridges:
    print(f"\n⚠️  CRITICAL: Found {len(bridges)} bridges:")
    for i, (u, v) in enumerate(sorted(bridges), 1):
        print(f"   {i}. {u} <-> {v}")

# Verify using NetworkX built-in function
nx_cut_vertices = set(nx.articulation_points(G))
nx_bridges = set(nx.bridges(G))

print(f"\n✅ Verification with NetworkX:")
print(f"   NetworkX cut-vertices: {len(nx_cut_vertices)}")
print(f"   NetworkX bridges: {len(nx_bridges)}")
print(f"   Our algorithm matches: {cut_vertices == nx_cut_vertices}")
print(f"   Our bridges match: {bridges == nx_bridges}")


### 3.1 Visualizing SPOF

Let's visualize the network highlighting cut-vertices and bridges to understand the vulnerabilities.


In [ ]:
def plot_spof_visualization(G, cut_vertices, bridges, figsize=(16, 12)):
    """
    Visualize the network with cut-vertices and bridges highlighted.
    """
    # Compute layout
    pos = nx.spring_layout(G, k=2, iterations=50, seed=42)
    
    plt.figure(figsize=figsize)
    
    # Draw all edges first (non-bridges in light gray)
    non_bridge_edges = [e for e in G.edges() if tuple(sorted(e)) not in bridges]
    bridge_edges = [e for e in G.edges() if tuple(sorted(e)) in bridges]
    
    nx.draw_networkx_edges(G, pos, edgelist=non_bridge_edges, 
                          alpha=0.2, width=0.5, edge_color='gray', style='solid')
    
    # Draw bridges in red and thicker
    if bridge_edges:
        nx.draw_networkx_edges(G, pos, edgelist=bridge_edges,
                              alpha=0.8, width=3, edge_color='red', style='solid',
                              label=f'Bridges ({len(bridges)})')
    
    # Draw non-cut-vertices
    non_cut_vertices = [n for n in G.nodes() if n not in cut_vertices]
    node_types = [G.nodes[n].get('node_type', 'unknown') for n in non_cut_vertices]
    
    type_to_color = {
        'core': '#FF6B6B',
        'server': '#4ECDC4',
        'workstation': '#95E1D3',
        'switch': '#F38181',
        'network': '#AA96DA',
        'device': '#FCBAD3'
    }
    
    # Draw nodes by type
    for node_type in set(node_types):
        nodes_of_type = [n for n in non_cut_vertices 
                        if G.nodes[n].get('node_type') == node_type]
        if nodes_of_type:
            nx.draw_networkx_nodes(
                G, pos,
                nodelist=nodes_of_type,
                node_size=300,
                node_color=type_to_color.get(node_type, '#CCCCCC'),
                alpha=0.6,
                label=node_type
            )
    
    # Draw cut-vertices in red with larger size
    if cut_vertices:
        nx.draw_networkx_nodes(
            G, pos,
            nodelist=list(cut_vertices),
            node_size=800,
            node_color='red',
            alpha=0.9,
            node_shape='s',  # Square shape for cut-vertices
            label=f'Cut-vertices ({len(cut_vertices)})'
        )
        # Label cut-vertices
        labels = {v: v for v in cut_vertices}
        nx.draw_networkx_labels(G, pos, labels, font_size=8, 
                               font_weight='bold', font_color='white')
    
    plt.title('Network Topology with Single Points of Failure\n'
              'Red squares = Cut-vertices, Red edges = Bridges',
              fontsize=16, fontweight='bold', pad=20)
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Visualize SPOF
print("📊 Visualizing Single Points of Failure...")
plot_spof_visualization(G, cut_vertices, bridges)


## Part IV: Cybersecurity Applications

Now that we understand centrality measures and SPOF, let's explore how they can be used in cybersecurity contexts.


### 4.1 Attacker's Perspective: Identifying High-Value Targets

**How hackers can use centrality measures:**

1. **Degree Centrality**: Identify highly connected nodes that provide access to many other systems
2. **Betweenness Centrality**: Find critical bottlenecks that control information flow
3. **Eigenvector Centrality**: Discover nodes connected to other important nodes (privilege escalation paths)
4. **PageRank**: Identify influential nodes that could provide maximum network access
5. **Cut-vertices**: Target nodes that, if compromised, could isolate parts of the network
6. **Bridges**: Target critical links that, if severed, could disrupt network operations


In [ ]:
# Identify high-value targets from attacker's perspective
print("🎯 ATTACKER'S PERSPECTIVE: High-Value Targets")
print("=" * 60)

# Combine multiple metrics to identify top targets
centrality_df['is_cut_vertex'] = centrality_df['node'].isin(cut_vertices)
centrality_df['threat_score'] = (
    centrality_df['betweenness'] * 0.3 +
    centrality_df['degree'] * 0.2 +
    centrality_df['eigenvector'] * 0.2 +
    centrality_df['pagerank'] * 0.15 +
    centrality_df['closeness'] * 0.15 +
    centrality_df['is_cut_vertex'].astype(int) * 0.1  # Bonus for cut-vertices
)

# Sort by threat score
top_targets = centrality_df.nlargest(15, 'threat_score')

print("\n🔥 TOP 15 HIGH-VALUE TARGETS (Attacker's Priority):")
print("-" * 60)
for idx, row in top_targets.iterrows():
    print(f"\n{top_targets.index.get_loc(idx) + 1}. {row['node']}")
    print(f"   Threat Score: {row['threat_score']:.4f}")
    print(f"   Type: {row['node_type']}, Department: {row['department']}")
    print(f"   Criticality: {row['criticality']}")
    print(f"   Cut-vertex: {'⚠️ YES' if row['is_cut_vertex'] else 'No'}")
    print(f"   Betweenness: {row['betweenness']:.4f}, Degree Count: {row['degree_count']}")
    print(f"   Attack Impact: ", end="")
    if row['is_cut_vertex']:
        print("HIGH - Network isolation possible")
    elif row['betweenness'] > 0.1:
        print("HIGH - Critical bottleneck")
    elif row['degree_count'] > 15:
        print("MEDIUM - High connectivity")
    else:
        print("LOW - Limited impact")

# Create visualization of attack targets
fig = go.Figure()

# Scatter plot: Threat score vs Betweenness
fig.add_trace(go.Scatter(
    x=centrality_df['betweenness'],
    y=centrality_df['threat_score'],
    mode='markers+text',
    text=centrality_df['node'],
    textposition="top center",
    textfont=dict(size=8),
    marker=dict(
        size=centrality_df['degree_count'] * 8 + 10,
        color=centrality_df['is_cut_vertex'].astype(int),
        colorscale=['#4ECDC4', 'red'],
        showscale=True,
        colorbar=dict(title="Is Cut-vertex", tickvals=[0, 1], ticktext=['No', 'Yes']),
        line=dict(width=2, color='white')
    ),
    hovertemplate='<b>%{text}</b><br>'
                  'Threat Score: %{y:.4f}<br>'
                  'Betweenness: %{x:.4f}<br>'
                  'Degree Count: %{marker.size:.0f}<extra></extra>',
    name='Nodes'
))

# Highlight top targets
for idx, row in top_targets.head(5).iterrows():
    fig.add_annotation(
        x=row['betweenness'],
        y=row['threat_score'],
        text=row['node'],
        showarrow=True,
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="red",
        ax=20,
        ay=-40,
        font=dict(size=10, color="red", family="Arial Black")
    )

fig.update_layout(
    title='Attack Target Analysis: Threat Score vs Betweenness Centrality<br>'
          '<sub>Size = Degree, Color = Cut-vertex (red), Top 5 targets annotated</sub>',
    xaxis_title='Betweenness Centrality',
    yaxis_title='Threat Score',
    height=700,
    plot_bgcolor='white'
)

fig.show()


### 4.2 Defender's Perspective: Protection Strategy

**How blue teams can use centrality measures:**

1. **Identify Critical Infrastructure**: Protect nodes with high centrality measures
2. **Redundancy Planning**: Add redundancy for cut-vertices and bridges
3. **Monitoring Priority**: Focus monitoring on high-centrality nodes
4. **Access Control**: Implement stricter access controls on critical nodes
5. **Network Segmentation**: Use centrality to plan network segmentation
6. **Incident Response**: Prioritize response based on node criticality


In [ ]:
# Defense strategy recommendations
print("🛡️ DEFENDER'S PERSPECTIVE: Protection Strategy")
print("=" * 60)

# Identify nodes that need protection
protection_priority = centrality_df.copy()
protection_priority['protection_priority'] = (
    protection_priority['threat_score'] * 0.5 +  # High threat = high priority
    protection_priority['is_cut_vertex'].astype(int) * 0.3 +  # Cut-vertices critical
    (protection_priority['criticality'] == 'critical').astype(int) * 0.2  # Marked critical
)

top_protect = protection_priority.nlargest(15, 'protection_priority')

print("\n🔒 TOP 15 NODES REQUIRING PROTECTION:")
print("-" * 60)

recommendations = []

for idx, row in top_protect.iterrows():
    node = row['node']
    node_type = row['node_type']
    is_cut = row['is_cut_vertex']
    
    print(f"\n{top_protect.index.get_loc(idx) + 1}. {node}")
    print(f"   Protection Priority: {row['protection_priority']:.4f}")
    print(f"   Type: {node_type}, Criticality: {row['criticality']}")
    
    # Generate recommendations
    recs = []
    
    if is_cut:
        recs.append("⚠️ CRITICAL: Add redundant connections (remove cut-vertex status)")
        recs.append("   - Implement load balancing")
        recs.append("   - Add backup systems")
    
    if row['betweenness'] > 0.1:
        recs.append("🔍 High monitoring priority (bottleneck)")
        recs.append("   - Implement 24/7 monitoring")
        recs.append("   - Set up alerts for anomalies")
    
    if node_type == 'core' or node_type == 'server':
        recs.append("🔐 Stricter access controls")
        recs.append("   - Multi-factor authentication")
        recs.append("   - Principle of least privilege")
        recs.append("   - Regular security audits")
    
    if row['degree_count'] > 15:
        recs.append("🌐 Network segmentation")
        recs.append("   - Isolate from less critical systems")
        recs.append("   - Implement firewall rules")
    
    recommendations.append({
        'node': node,
        'priority': row['protection_priority'],
        'recommendations': recs
    })
    
    for rec in recs[:3]:  # Show first 3 recommendations
        print(f"   {rec}")

# Create protection priority visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Protection Priority Ranking', 'Cut-vertices Analysis',
                   'Betweenness vs Protection Priority', 'Node Type Protection Needs'),
    specs=[[{"type": "bar"}, {"type": "bar"}],
           [{"type": "scatter"}, {"type": "bar"}]]
)

# Protection priority bar chart
top_15_protect = protection_priority.nlargest(15, 'protection_priority')
fig.add_trace(
    go.Bar(x=top_15_protect['node'], y=top_15_protect['protection_priority'],
           name='Protection Priority', marker_color='#FF6B6B',
           text=top_15_protect['protection_priority'].round(3),
           textposition='outside'),
    row=1, col=1
)

# Cut-vertices analysis
cut_vertex_df = centrality_df[centrality_df['is_cut_vertex']].copy()
if len(cut_vertex_df) > 0:
    fig.add_trace(
        go.Bar(x=cut_vertex_df['node'], y=cut_vertex_df['betweenness'],
               name='Cut-vertices Betweenness', marker_color='red',
               text=cut_vertex_df['betweenness'].round(3),
               textposition='outside'),
        row=1, col=2
    )

# Scatter: Betweenness vs Protection Priority
fig.add_trace(
    go.Scatter(
        x=protection_priority['betweenness'],
        y=protection_priority['protection_priority'],
        mode='markers',
        marker=dict(
            size=protection_priority['degree_count'] * 5 + 5,
            color=protection_priority['is_cut_vertex'].astype(int),
            colorscale=['#4ECDC4', 'red'],
            showscale=True,
            colorbar=dict(title="Cut-vertex", x=1.15, tickvals=[0, 1], ticktext=['No', 'Yes']),
            line=dict(width=1, color='white')
        ),
        text=protection_priority['node'],
        hovertemplate='<b>%{text}</b><br>'
                      'Protection Priority: %{y:.4f}<br>'
                      'Betweenness: %{x:.4f}<extra></extra>',
        name='Nodes'
    ),
    row=2, col=1
)

# Node type protection needs
type_protection = protection_priority.groupby('node_type')['protection_priority'].mean().sort_values(ascending=False)
fig.add_trace(
    go.Bar(x=type_protection.index, y=type_protection.values,
           name='Avg Protection Priority by Type', marker_color='#AA96DA',
           text=type_protection.values.round(3),
           textposition='outside'),
    row=2, col=2
)

# Update layout
for i in range(1, 3):
    for j in range(1, 3):
        fig.update_xaxes(tickangle=-45, row=i, col=j)

fig.update_layout(
    height=1000,
    showlegend=False,
    title_text="Defense Strategy Analysis",
    title_x=0.5
)

fig.update_xaxes(title_text="Node", row=2, col=1)
fig.update_yaxes(title_text="Protection Priority", row=2, col=1)
fig.update_xaxes(title_text="Betweenness", row=2, col=1)

fig.show()


### 4.3 Attack Path Analysis

Let's analyze potential attack paths using centrality measures. An attacker might:
1. Start from a low-security node (workstation)
2. Move through high-betweenness nodes (bottlenecks)
3. Target cut-vertices to isolate network segments
4. Reach high-value targets (servers, domain controllers)


In [ ]:
# Simulate attack paths
def find_attack_paths(G, start_node, target_node, max_paths=5):
    """
    Find potential attack paths from a starting node to a target node.
    Prioritizes paths through high-betweenness nodes and cut-vertices.
    """
    try:
        # Find all simple paths
        paths = list(nx.all_simple_paths(G, start_node, target_node, cutoff=10))[:max_paths]
        
        # Score paths based on vulnerability
        scored_paths = []
        for path in paths:
            score = 0
            cut_vertices_in_path = sum(1 for n in path if n in cut_vertices)
            high_betweenness_nodes = sum(1 for n in path 
                                        if centrality_df[centrality_df['node'] == n]['betweenness'].values[0] > 0.1)
            
            # Paths through cut-vertices are more dangerous
            score += cut_vertices_in_path * 10
            # Paths through high-betweenness nodes are more likely
            score += high_betweenness_nodes * 5
            # Shorter paths are more likely
            score += (11 - len(path)) * 2
            
            scored_paths.append((path, score, cut_vertices_in_path, high_betweenness_nodes))
        
        # Sort by score (higher is more dangerous/likely)
        scored_paths.sort(key=lambda x: x[1], reverse=True)
        return scored_paths
    except nx.NetworkXNoPath:
        return []

# Find attack paths from a workstation to a domain controller
workstations = [n for n in G.nodes() if 'Workstation' in n and G.nodes[n].get('node_type') == 'workstation']
domain_controllers = [n for n in G.nodes() if 'Domain-Controller' in n]

if workstations and domain_controllers:
    start = workstations[0]  # Attacker's entry point
    target = domain_controllers[0]  # High-value target
    
    print(f"🎯 ATTACK PATH ANALYSIS")
    print("=" * 60)
    print(f"From: {start} (Potential entry point)")
    print(f"To: {target} (High-value target: Domain Controller)")
    print("-" * 60)
    
    attack_paths = find_attack_paths(G, start, target, max_paths=5)
    
    if attack_paths:
        print(f"\nFound {len(attack_paths)} potential attack paths:\n")
        for i, (path, score, cut_count, high_bet_count) in enumerate(attack_paths, 1):
            print(f"Path {i} (Danger Score: {score}):")
            print(f"  {' -> '.join(path)}")
            print(f"  Length: {len(path) - 1} hops")
            print(f"  Cut-vertices: {cut_count}, High-betweenness nodes: {high_bet_count}")
            print()
    else:
        print("\nNo direct path found (network segmentation working!)")
    
    # Visualize attack path
    if attack_paths:
        best_path = attack_paths[0][0]
        
        # Create visualization
        pos = nx.spring_layout(G, k=2, iterations=50, seed=42)
        
        plt.figure(figsize=(16, 12))
        
        # Draw all edges
        nx.draw_networkx_edges(G, pos, alpha=0.1, width=0.3, edge_color='gray')
        
        # Highlight attack path
        path_edges = [(best_path[i], best_path[i+1]) for i in range(len(best_path)-1)]
        nx.draw_networkx_edges(G, pos, edgelist=path_edges, 
                              alpha=0.8, width=4, edge_color='red', style='dashed',
                              label='Attack Path')
        
        # Draw all nodes
        nx.draw_networkx_nodes(G, pos, node_size=200, node_color='lightblue', alpha=0.6)
        
        # Highlight path nodes
        nx.draw_networkx_nodes(G, pos, nodelist=best_path,
                              node_size=500, node_color='red', alpha=0.8,
                              label='Path Nodes')
        
        # Highlight start and target
        nx.draw_networkx_nodes(G, pos, nodelist=[start],
                              node_size=800, node_color='orange', alpha=0.9,
                              node_shape='s', label='Start (Entry)')
        nx.draw_networkx_nodes(G, pos, nodelist=[target],
                              node_size=800, node_color='darkred', alpha=0.9,
                              node_shape='s', label='Target (Domain Controller)')
        
        # Label important nodes
        labels = {start: 'START', target: 'TARGET'}
        labels.update({n: n for n in best_path if n in cut_vertices})
        nx.draw_networkx_labels(G, pos, labels, font_size=8, font_weight='bold')
        
        plt.title(f'Attack Path Analysis: {start} → {target}\n'
                 f'Path through {len(best_path)-1} hops, Danger Score: {attack_paths[0][1]}',
                 fontsize=16, fontweight='bold')
        plt.legend(loc='upper left')
        plt.axis('off')
        plt.tight_layout()
        plt.show()
else:
    print("⚠️ Could not find suitable nodes for attack path analysis")


## Part V: Summary and Key Insights

### Key Takeaways

1. **Centrality Measures** help identify critical nodes from different perspectives:
   - **Degree**: Highly connected nodes (many attack surfaces)
   - **Betweenness**: Network bottlenecks (critical for information flow)
   - **Eigenvector**: Nodes connected to important nodes (privilege escalation)
   - **PageRank**: Influential nodes (maximum network access)
   - **Closeness**: Centrally located nodes (fast propagation)

2. **Single Points of Failure (SPOF)**:
   - **Cut-vertices**: Removing these nodes disconnects the network
   - **Bridges**: Removing these edges disconnects the network
   - Critical for both attackers (targets) and defenders (protection priority)

3. **Cybersecurity Applications**:
   - **Attackers**: Use centrality to identify high-value targets and attack paths
   - **Defenders**: Use centrality to prioritize protection and monitoring
   - **Network Design**: Identify and eliminate SPOF through redundancy

### Recommendations

1. **For Network Administrators**:
   - Identify all cut-vertices and bridges
   - Add redundancy to eliminate SPOF
   - Implement monitoring on high-centrality nodes
   - Use network segmentation based on centrality analysis

2. **For Security Teams**:
   - Prioritize protection of high-centrality nodes
   - Monitor attack paths through cut-vertices
   - Implement stricter access controls on critical nodes
   - Regular centrality analysis to identify new vulnerabilities

3. **For Incident Response**:
   - Respond first to incidents on high-centrality nodes
   - Isolate cut-vertices if compromised
   - Trace attack paths using centrality measures
   - Use betweenness to identify lateral movement


In [ ]:
# Create a comprehensive summary dashboard
print("📊 NETWORK SECURITY SUMMARY")
print("=" * 60)

print(f"\n📈 Network Statistics:")
print(f"   Total Nodes: {G.number_of_nodes()}")
print(f"   Total Edges: {G.number_of_edges()}")
print(f"   Network Density: {nx.density(G):.4f}")
print(f"   Average Clustering: {nx.average_clustering(G):.4f}")
print(f"   Diameter: {nx.diameter(G) if nx.is_connected(G) else 'N/A (disconnected)'}")

print(f"\n⚠️  Vulnerability Assessment:")
print(f"   Cut-vertices (SPOF): {len(cut_vertices)}")
print(f"   Bridges (SPOF): {len(bridges)}")
print(f"   High-betweenness nodes (>0.1): {len(centrality_df[centrality_df['betweenness'] > 0.1])}")
print(f"   High-degree nodes (>15): {len(centrality_df[centrality_df['degree_count'] > 15])}")

print(f"\n🎯 Top 5 Attack Targets:")
top_targets_summary = centrality_df.nlargest(5, 'threat_score')
for i, (idx, row) in enumerate(top_targets_summary.iterrows(), 1):
    print(f"   {i}. {row['node']} (Threat Score: {row['threat_score']:.4f})")

print(f"\n🛡️  Top 5 Protection Priorities:")
top_protect_summary = protection_priority.nlargest(5, 'protection_priority')
for i, (idx, row) in enumerate(top_protect_summary.iterrows(), 1):
    print(f"   {i}. {row['node']} (Priority: {row['protection_priority']:.4f})")

print(f"\n💡 Key Recommendations:")
print(f"   1. Add redundancy to {len(cut_vertices)} cut-vertices")
print(f"   2. Protect {len(bridges)} bridge connections")
print(f"   3. Implement monitoring on top {len(top_targets_summary)} high-value targets")
print(f"   4. Review network segmentation around high-betweenness nodes")
print(f"   5. Regular centrality analysis to detect topology changes")

# Create final summary visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Network Vulnerability Map', 'Centrality Distribution',
                   'Node Type Risk Analysis', 'Protection vs Threat Score'),
    specs=[[{"type": "scatter"}, {"type": "histogram"}],
           [{"type": "bar"}, {"type": "scatter"}]]
)

# Vulnerability map: Betweenness vs Degree Count, colored by cut-vertex status
fig.add_trace(
    go.Scatter(
        x=centrality_df['degree_count'],
        y=centrality_df['betweenness'],
        mode='markers',
        marker=dict(
            size=10,
            color=centrality_df['is_cut_vertex'].astype(int),
            colorscale=['#4ECDC4', 'red'],
            showscale=True,
            colorbar=dict(title="Cut-vertex", x=1.02, tickvals=[0, 1], ticktext=['No', 'Yes']),
            line=dict(width=1, color='white')
        ),
        text=centrality_df['node'],
        hovertemplate='<b>%{text}</b><br>Degree Count: %{x}<br>Betweenness: %{y:.4f}<extra></extra>',
        name='Nodes'
    ),
    row=1, col=1
)

# Betweenness distribution
fig.add_trace(
    go.Histogram(x=centrality_df['betweenness'], nbinsx=20, name='Betweenness',
                marker_color='#4ECDC4'),
    row=1, col=2
)

# Node type risk analysis
type_risk = centrality_df.groupby('node_type').agg({
    'threat_score': 'mean',
    'is_cut_vertex': 'sum'
}).sort_values('threat_score', ascending=False)

fig.add_trace(
    go.Bar(x=type_risk.index, y=type_risk['threat_score'],
           name='Avg Threat Score', marker_color='#FF6B6B',
           text=type_risk['threat_score'].round(3),
           textposition='outside'),
    row=2, col=1
)

# Protection vs Threat
fig.add_trace(
    go.Scatter(
        x=protection_priority['threat_score'],
        y=protection_priority['protection_priority'],
        mode='markers',
        marker=dict(
            size=8,
            color=protection_priority['is_cut_vertex'].astype(int),
            colorscale=['#95E1D3', 'red'],
            showscale=False,
            line=dict(width=1, color='white')
        ),
        text=protection_priority['node'],
        hovertemplate='<b>%{text}</b><br>Threat: %{x:.4f}<br>Protection: %{y:.4f}<extra></extra>',
        name='Nodes'
    ),
    row=2, col=2
)

# Update layout
fig.update_xaxes(title_text="Degree Count", row=1, col=1)
fig.update_yaxes(title_text="Betweenness", row=1, col=1)
fig.update_xaxes(title_text="Betweenness Centrality", row=1, col=2)
fig.update_yaxes(title_text="Frequency", row=1, col=2)
fig.update_xaxes(title_text="Node Type", row=2, col=1)
fig.update_yaxes(title_text="Average Threat Score", row=2, col=1)
fig.update_xaxes(title_text="Threat Score", row=2, col=2)
fig.update_yaxes(title_text="Protection Priority", row=2, col=2)

fig.update_layout(
    height=1000,
    showlegend=False,
    title_text="Network Security Analysis Dashboard",
    title_x=0.5
)

fig.show()

print("\n✅ Analysis complete! Review the visualizations above for detailed insights.")
